In [ ]:
!pip install langgraph langchain ipython langchain_nvidia_ai_endpoints langchain_community dotenv typing chromadb langchain_chroma

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 78.6/78.6 kB 2.1 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.3/67.3 kB 5.2 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.7/43.7 kB 2.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 154.9/154.9 kB 7.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 41.6/41.6 kB 2.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 43.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 19.0/19.0 MB 95.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 94.9/94.9 kB 7.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 284.2/284.2 kB 17.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 62.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

In [ ]:
!pip install langchain-tavily


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.3/12.3 MB 61.0 MB/s eta 0:00:00


In [ ]:
!pip install agentql

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 65.5/65.5 kB 2.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 45.1/45.1 MB 22.2 MB/s eta 0:00:00


In [ ]:
# Import essential libraries
from typing import TypedDict, List, Sequence, Annotated
from langgraph.graph import StateGraph, START, END
from langchain_core.messages import HumanMessage, AIMessage, SystemMessage, ToolMessage, BaseMessage
from langgraph.graph.message import add_messages
from google.colab import userdata
from langchain_nvidia_ai_endpoints import ChatNVIDIA, NVIDIAEmbeddings
from langchain_core.prompts import PromptTemplate

In [ ]:
# Initialize our LLM
nvidia_api_key = userdata.get("NVIDIA_API_KEY")
llm = ChatNVIDIA(model="meta/llama3-70b-instruct", api_key=nvidia_api_key)

print("✅ Setup complete! Let's start building agents!")

✅ Setup complete! Let's start building agents!


In [ ]:
from typing import TypedDict, List, Sequence, Annotated
from langgraph.graph import StateGraph, START, END
from langchain_core.messages import HumanMessage, AIMessage, SystemMessage, ToolMessage, BaseMessage
from langgraph.graph.message import add_messages
from google.colab import userdata
from langchain_nvidia_ai_endpoints import ChatNVIDIA
from langchain_tavily import TavilySearch
import agentql
import os

# Initialize LLM and APIs
nvidia_api_key = userdata.get("NVIDIA_API_KEY")
llm = ChatNVIDIA(model="meta/llama3-70b-instruct", api_key=nvidia_api_key)
tavily_api_key = userdata.get("TAVILY_API_KEY")
agentql_api_key = userdata.get("AGENTQL_API_KEY")

os.environ["TAVILY_API_KEY"] = tavily_api_key
os.environ["AGENTQL_API_KEY"] = agentql_api_key


# Initialize tools
tavily_tool = TavilySearch(
    max_results=5,
    include_domains=["amazon.in", "flipkart.com", "croma.com", "nykaa.com", "meesho.com"],
    search_depth="advanced"
)


class ConversationState(TypedDict):
    messages: Annotated[Sequence[BaseMessage], add_messages]
    user_name: str
    conversation_summary: str

def extract_prices(state: ConversationState) -> dict:
    """Extract prices using AgentQL from specific e-commerce sites"""
    product = state["messages"][-1].content
    sites = {
        "amazon.in": "{ products[] { title price }}",
        "flipkart.com": "{ products[] { name price }}",
        "croma.com": "{ items[] { description price }}"
    }
    prices = {}
    for site, query in sites.items():
        try:
            data = agentql.query(
                url=f"https://{site}/search?q={product}",
                query=query
            )
            products = data.get("products", []) if "products" in data else []
            prices[site] = [
                {"name": item.get("name", item.get("title", "Unknown")), "price": item.get("price", "N/A")}
                for item in products
            ][:3]
        except Exception as e:
            prices[site] = f"Error retrieving data: {str(e)}"
    return prices

def price_comparison_node(state: ConversationState) -> ConversationState:
    """Handle price comparison requests"""
    user_query = state["messages"][-1].content.lower()
    tavily_results = tavily_tool.invoke({
        "query": f"{user_query} price comparison india",
        "max_results": 5
    })
    direct_prices = extract_prices(state)
    response = ["Here are the best prices I found:"]
    for site, results in direct_prices.items():
        if isinstance(results, list):
            response.append(f"**{site.upper()}**")
            for item in results:
                response.append(f"- {item['name']}: ₹{item['price']}")
    response.append("\n**Other Options from Web Search:**")
    for result in tavily_results["results"][:3]:
        response.append(f"- [{result['title']}]({result['url']}): {result['content']}")
    return {"messages": [AIMessage(content="\n".join(response))]}

def chat_node(state: ConversationState) -> ConversationState:
    """Main conversation handler"""
    user_message = state["messages"][-1].content
    if "price" in user_message or "compare" in user_message:
        return price_comparison_node(state)
    system_prompt = f"""You are a shopping assistant helping {state['user_name']} find best deals.
    Focus on price comparisons across Indian e-commerce platforms."""
    messages = [SystemMessage(content=system_prompt)] + state["messages"]
    response = llm.invoke(messages)
    return {"messages": [response]}

# Build conversation graph
price_bot = StateGraph(ConversationState)
price_bot.add_node("chat_node", chat_node)
price_bot.add_edge(START, "chat_node")
price_bot.add_edge("chat_node", END)
price_agent = price_bot.compile()

# Example usage
def compare_prices(product: str):
    state = {
        "messages": [HumanMessage(content=product)],
        "user_name": "User",
        "conversation_summary": ""
    }
    return price_agent.invoke(state)

# Test
result = compare_prices("iPhone 15 128GB")
print(result["messages"][-1].content)


You're looking for the iPhone 15 128GB model!

Let me help you find the best deals across Indian e-commerce platforms. Here are some prices I've found:

1. **Amazon India**: ₹79,990 (Free delivery, 1-year warranty)
2. **Flipkart**: ₹79,999 (Free delivery, 1-year warranty)
3. **Apple India Store**: ₹79,990 (Free delivery, 1-year warranty)
4. **Paytm Mall**: ₹78,499 (Free delivery, 1-year warranty, with cashback offers)
5. **Tata CLiQ**: ₹79,990 (Free delivery, 1-year warranty)

As you can see, Paytm Mall is offering the lowest price with a cashback offer. However, please note that prices may vary depending on the color and storage variant you choose.

If you're looking for additional discounts or offers, here are some tips:

* Check for cashback offers on credit/debit cards or wallets like Paytm, Amazon Pay, or Flipkart's Axis Bank card.
* Look for exchange offers or discounts on old iPhone models.
* Consider purchasing a refurbished or certified pre-owned iPhone 15 for a lower price.



In [ ]:
from typing import TypedDict, Sequence, Annotated
from langgraph.graph import StateGraph, START, END
from langchain_core.messages import HumanMessage, AIMessage, SystemMessage, BaseMessage
from langgraph.graph.message import add_messages
from langchain_core.tools import tool
from langchain_nvidia_ai_endpoints import ChatNVIDIA
from langchain_tavily import TavilySearch
from langgraph.prebuilt import ToolNode
import agentql
import os

from google.colab import userdata
nvidia_api_key = userdata.get("NVIDIA_API_KEY")
tavily_api_key = userdata.get("TAVILY_API_KEY")
agentql_api_key = userdata.get("AGENTQL_API_KEY")

os.environ["TAVILY_API_KEY"] = tavily_api_key
os.environ["AGENTQL_API_KEY"] = agentql_api_key


llm = ChatNVIDIA(model="meta/llama3-70b-instruct", api_key=nvidia_api_key)
tavily_tool = TavilySearch(
    max_results=5,
    include_domains=["amazon.in", "flipkart.com", "croma.com"],
    search_depth="advanced"
)

def is_relevant(name: str, query: str, threshold: float = 0.6) -> bool:
    from difflib import SequenceMatcher
    name_lower = name.lower()
    query_lower = query.lower()
    return SequenceMatcher(None, name_lower, query_lower).ratio() >= threshold


class ReActState(TypedDict):
    messages: Annotated[Sequence[BaseMessage], add_messages]

#Price comparison via AgentQL
@tool
def compare_prices(product_query: str) -> str:
    """
    Compare prices of a product across Amazon.in, Flipkart, and Croma using AgentQL.
    """
    product = product_query.lower()
    sites = {
        "www.amazon.in": "{ products[] { title price } }",
        "www.flipkart.com": "{ products[] { name price } }",
        "www.croma.com": "{ items[] { description price } }"
    }

    prices = []
    for site, query in sites.items():
        try:
            data = agentql.query(url=f"https://{site}/search?q={product}", query=query)
            key = "products" if "products" in data else "items"
            items = data.get(key, [])

            for item in items:
                name = item.get("name") or item.get("title") or item.get("description", "")
                price = item.get("price")
                if price and is_relevant(name, product):
                    prices.append(f"- {name.title()} on **{site}**: ₹{price}")
                    if len(prices) >= 9:
                        break
        except Exception:
            continue

    if prices:
        return "### 🛍️ Price Comparison Results\n" + "\n".join(prices)
    else:
        # Fallback to Tavily search results if AgentQL fails
        results = tavily_tool.invoke({
            "query": f"{product_query} price comparison india",
            "max_results": 5
        }).get("results", [])[:3]
        if results:
            fallback_links = "\n".join(
                [f"- [{r['title']}]({r['url']})" for r in results if r.get("title") and r.get("url")]
            )
            return "No prices found via AgentQL. Try these links:\n" + fallback_links
        else:
            return "Sorry, no price information could be found."


llm_with_tools = llm.bind_tools([compare_prices])

# Reasoning node for ReAct pattern
def reasoning_node(state: ReActState) -> ReActState:
    system_prompt = SystemMessage(content=(
        "You are a smart shopping assistant that helps users compare prices "
        "across Amazon.in, Flipkart, and Croma. Use the `compare_prices` tool "
        "to fetch up-to-date price info using AgentQL. If that fails, fall back to Tavily."
    ))
    messages = [system_prompt] + state["messages"]

    print("🧠 Reasoning with user query...")
    response = llm_with_tools.invoke(messages)
    # print("🤖 Assistant:", response.content)
    return {"messages": [response]}

# Decide next step in ReAct graph
def should_continue(state: ReActState) -> str:
    last_msg = state["messages"][-1]
    if hasattr(last_msg, "tool_calls") and last_msg.tool_calls:
        print("🔧 Tool use required.")
        return "tool_use"
    else:
        print("✅ Final answer ready.")
        return "final"

# Tool execution node
tools_node = ToolNode([compare_prices])

# Build state graph
graph = StateGraph(ReActState)
graph.add_node("reasoning", reasoning_node)
graph.add_node("tool_use", tools_node)
graph.add_edge(START, "reasoning")
graph.add_conditional_edges("reasoning", should_continue, {
    "tool_use": "tool_use",
    "final": END
})
graph.add_edge("tool_use", "reasoning")
price_compare_app = graph.compile()

# Interface function to chat with bot
def ask_bot(query: str) -> str:
    state = {"messages": [HumanMessage(content=query)]}
    return price_compare_app.invoke(state)["messages"][-1].content

# Test loop example
if __name__ == "__main__":
    queries = ["HP Pavillion",
        "realme 6",
        "Samsung Galaxy M14"
    ]
    for q in queries:
        print(f"\n🧑 User: {q}")
        response = ask_bot(q)
        print(f"🤖 Bot:\n{response}")


/usr/local/lib/python3.11/dist-packages/langchain_nvidia_ai_endpoints/chat_models.py:592: UserWarning: Model 'meta/llama3-70b-instruct' is not known to support tools. Your tool binding may fail at inference time.
  warnings.warn(



🧑 User: HP Pavillion
🧠 Reasoning with user query...
✅ Final answer ready.
🤖 Bot:
I'd be happy to help you compare prices for HP Pavilion laptops across Amazon.in, Flipkart, and Croma!

Here are the current prices:

**HP Pavilion Gaming 15t (2022)**
* Amazon.in: ₹73,990 (available in 3-4 days) [Check Price](https://www.amazon.in/HP-Pavilion-Gaming-15t-Laptop/dp/B09R8KRYGK)
* Flipkart: ₹75,990 (available in 2-3 days) [Check Price](https://www.flipkart.com/hp-pavilion-gaming-15t-laptop/p/itm4c1ebee55cacf)
* Croma: ₹76,990 (available in 2-3 days) [Check Price](https://www.croma.com/hp-pavilion-gaming-15t-laptop-/p/230241)

**HP Pavilion x360 (2022)**
* Amazon.in: ₹64,990 (available in 3-4 days) [Check Price](https://www.amazon.in/HP-Pavilion-x360-Laptop-14-ek1026tu/dp/B09R9FHF2D)
* Flipkart: ₹66,990 (available in 2-3 days) [Check Price](https://www.flipkart.com/hp-pavilion-x360-laptop/p/itm2e5ebee55cacf)
* Croma: ₹67,990 (available in 2-3 days) [Check Price](https://www.croma.com/hp-pavil

In [ ]:
from typing import TypedDict, Sequence, Annotated
from langgraph.graph import StateGraph, START, END
from langchain_core.messages import HumanMessage, AIMessage, SystemMessage, BaseMessage
from langgraph.graph.message import add_messages
from langchain_core.tools import tool
from langchain_nvidia_ai_endpoints import ChatNVIDIA
from langchain_tavily import TavilySearch
from langgraph.prebuilt import ToolNode
import agentql
import os

from google.colab import userdata
nvidia_api_key = userdata.get("NVIDIA_API_KEY")
tavily_api_key = userdata.get("TAVILY_API_KEY")
agentql_api_key = userdata.get("AGENTQL_API_KEY")

os.environ["TAVILY_API_KEY"] = tavily_api_key
os.environ["AGENTQL_API_KEY"] = agentql_api_key


llm = ChatNVIDIA(model="meta/llama3-70b-instruct", api_key=nvidia_api_key)
tavily_tool = TavilySearch(
    max_results=5,
    include_domains=["amazon.in", "flipkart.com", "croma.com"],
    search_depth="advanced"
)

def is_relevant(name: str, query: str, threshold: float = 0.6) -> bool:
    from difflib import SequenceMatcher
    name_lower = name.lower()
    query_lower = query.lower()
    return SequenceMatcher(None, name_lower, query_lower).ratio() >= threshold


class ReActState(TypedDict):
    messages: Annotated[Sequence[BaseMessage], add_messages]

#Price comparison via AgentQL
@tool
def compare_prices(product_query: str) -> str:
    """
    Compare prices of a product across Amazon.in, Flipkart, and Croma using AgentQL.
    """
    product = product_query.lower()
    sites = {
        "www.amazon.in": "{ products[] { title price } }",
        "www.flipkart.com": "{ products[] { name price } }",
        "www.croma.com": "{ items[] { description price } }"
    }

    prices = []
    for site, query in sites.items():
        try:
            data = agentql.query(url=f"https://{site}/search?q={product}", query=query)
            key = "products" if "products" in data else "items"
            items = data.get(key, [])

            for item in items:
                name = item.get("name") or item.get("title") or item.get("description", "")
                price = item.get("price")
                if price and is_relevant(name, product):
                    prices.append(f"- {name.title()} on **{site}**: ₹{price}")
                    if len(prices) >= 9:
                        break
        except Exception:
            continue

    if prices:
        return "### 🛍️ Price Comparison Results\n" + "\n".join(prices)
    else:
        # Fallback to Tavily search results if AgentQL fails
        results = tavily_tool.invoke({
            "query": f"{product_query} price comparison india",
            "max_results": 5
        }).get("results", [])[:3]
        if results:
            fallback_links = "\n".join(
                [f"- [{r['title']}]({r['url']})" for r in results if r.get("title") and r.get("url")]
            )
            return "No prices found via AgentQL. Try these links:\n" + fallback_links
        else:
            return "Sorry, no price information could be found."


llm_with_tools = llm.bind_tools([compare_prices])

# Reasoning node for ReAct pattern
def reasoning_node(state: ReActState) -> ReActState:
    system_prompt = SystemMessage(content=(
        "You are a smart shopping assistant that helps users compare prices "
        "across Amazon.in, Flipkart, and Croma. Use the `compare_prices` tool "
        "to fetch up-to-date price info using AgentQL. If that fails, fall back to Tavily."
    ))
    messages = [system_prompt] + state["messages"]

    print("🧠 Reasoning with user query...")
    response = llm_with_tools.invoke(messages)
    # print("🤖 Assistant:", response.content)
    return {"messages": [response]}

# Decide next step in ReAct graph
def should_continue(state: ReActState) -> str:
    last_msg = state["messages"][-1]
    if hasattr(last_msg, "tool_calls") and last_msg.tool_calls:
        print("🔧 Tool use required.")
        return "tool_use"
    else:
        print("✅ Final answer ready.")
        return "final"

# Tool execution node
tools_node = ToolNode([compare_prices])

# Build state graph
graph = StateGraph(ReActState)
graph.add_node("reasoning", reasoning_node)
graph.add_node("tool_use", tools_node)
graph.add_edge(START, "reasoning")
graph.add_conditional_edges("reasoning", should_continue, {
    "tool_use": "tool_use",
    "final": END
})
graph.add_edge("tool_use", "reasoning")
price_compare_app = graph.compile()

# Interface function to chat with bot
def ask_bot(query: str) -> str:
    state = {"messages": [HumanMessage(content=query)]}
    return price_compare_app.invoke(state)["messages"][-1].content

# Test loop example
if __name__ == "__main__":
    queries = ["HP Pavillion",
        "realme 6",
        "Samsung Galaxy M14",
        "Boat airpods",
               "what variants of boat airdopes are available and compare their prices?"
    ]
    for q in queries:
        print(f"\n🧑 User: {q}")
        response = ask_bot(q)
        print(f"🤖 Bot:\n{response}")


/usr/local/lib/python3.11/dist-packages/langchain_nvidia_ai_endpoints/chat_models.py:592: UserWarning: Model 'meta/llama3-70b-instruct' is not known to support tools. Your tool binding may fail at inference time.
  warnings.warn(



🧑 User: HP Pavillion
🧠 Reasoning with user query...
✅ Final answer ready.
🤖 Bot:
I'd be happy to help you compare prices for HP Pavilion laptops across Amazon.in, Flipkart, and Croma!

Please note that the prices may vary depending on the model, configuration, and availability. Here are the search results for HP Pavilion laptops:

**HP Pavilion Gaming 15 (2021) - Core i5, 8GB RAM, 1TB HDD, NVIDIA GeForce GTX 1650**

* Amazon.in: ₹63,990 (available)
* Flipkart: ₹64,990 (available)
* Croma: ₹65,990 (available)

**HP Pavilion Gaming 15 (2022) - Core i7, 16GB RAM, 512GB SSD, NVIDIA GeForce RTX 2060**

* Amazon.in: ₹1,09,990 (available)
* Flipkart: ₹1,12,990 (available)
* Croma: ₹1,14,990 (available)

**HP Pavilion Aero 13 (2022) - Core i5, 16GB RAM, 512GB SSD, Intel Iris Xe Graphics**

* Amazon.in: ₹74,990 (available)
* Flipkart: ₹76,990 (available)
* Croma: ₹78,990 (available)

Please note that prices may vary based on your location and availability. I'll keep updating the prices in real